<a href="https://colab.research.google.com/github/LeksaAura/dataset.ipynb/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np

In [49]:
movies_df = pd.read_csv("/content/movies.csv",
                        usecols=['movieId', 'title'],
                        dtype={'movieId': 'int32', 'title': 'str'})

In [50]:
rating_df = pd.read_csv(r"/content/ratings.csv",
                        usecols=['userId', 'movieId', 'rating'],
                        dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [51]:
rating_df.head(10)

,userId,movieId,rating
0,1,1,4.000
1,1,3,4.000
2,1,6,4.000
3,1,47,5.000
4,1,50,5.000
5,1,70,3.000
6,1,101,5.000
7,1,110,4.000
8,1,151,5.000
9,1,157,5.000


In [52]:
movies_df.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [53]:
movies_df.shape

(9742, 2)

In [54]:
rating_df.shape

(100836, 3)

In [55]:
df = pd.merge(rating_df, movies_df, on = 'movieId')
df

,userId,movieId,rating,title
0,1,1,4.000,Toy Story (1995)
1,1,3,4.000,Grumpier Old Men (1995)
2,1,6,4.000,Heat (1995)
3,1,47,5.000,Seven (a.k.a. Se7en) (1995)
4,1,50,5.000,"Usual Suspects, The (1995)"
...,...,...,...,...
100831,610,166534,4.000,Split (2017)
100832,610,168248,5.000,John Wick: Chapter Two (2017)
100833,610,168250,5.000,Get Out (2017)
100834,610,168252,5.000,Logan (2017)


In [56]:
combine_movie_rating = df.dropna(axis=0, subset=['title'])

movie_ratingCount = (combine_movie_rating.
                     groupby(by=['title'])['rating'].
                     count().
                     reset_index().
                     rename(columns={'rating': 'totalRatingCount'})
                     [['title', 'totalRatingCount']]
                    )

movie_ratingCount

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [57]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on='title', right_on='title', how='left')
rating_with_totalRatingCount.head(10)

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,1,3,4.000,Grumpier Old Men (1995),52
2,1,6,4.000,Heat (1995),102
3,1,47,5.000,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.000,"Usual Suspects, The (1995)",204
5,1,70,3.000,From Dusk Till Dawn (1996),55
6,1,101,5.000,Bottle Rocket (1996),23
7,1,110,4.000,Braveheart (1995),237
8,1,151,5.000,Rob Roy (1995),44
9,1,157,5.000,Canadian Bacon (1995),11


In [58]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [59]:
 popularity_threshold = 75 # sebelumnya 50
 rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
 rating_popular_movie.head(10)

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
2,1,6,4.000,Heat (1995),102
3,1,47,5.000,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.000,"Usual Suspects, The (1995)",204
7,1,110,4.000,Braveheart (1995),237
12,1,223,3.000,Clerks (1994),104
13,1,231,5.000,Dumb & Dumber (Dumb and Dumber) (1994),133
15,1,260,5.000,Star Wars: Episode IV - A New Hope (1977),251
16,1,296,3.000,Pulp Fiction (1994),307
17,1,316,3.000,Stargate (1994),140


In [60]:
rating_popular_movie.shape

(28563, 5)

In [61]:
movie_feature_df=rating_popular_movie.pivot_table(index='title', columns='userId', values='rating').fillna(0)
movie_feature_df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000
Addams Family Values (1993),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,4.000,0.000,4.000,0.000,0.000,0.000,2.500,0.000,0.000
Airplane! (1980),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,5.000,0.000,4.000,0.000,0.000,3.500
Aladdin (1992),0.000,0.000,0.000,4.000,4.000,5.000,3.000,0.000,0.000,4.000,...,0.000,0.000,0.000,3.000,3.500,0.000,0.000,3.000,0.000,0.000
Alien (1979),4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,4.000,3.000,4.000,0.000,4.500
Aliens (1986),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,3.500,0.000,4.500,0.000,5.000


In [62]:
!pip install scipy
!pip install scikit-learn

In [63]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_feature_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [69]:
movie_feature_df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000
Addams Family Values (1993),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,4.000,0.000,4.000,0.000,0.000,0.000,2.500,0.000,0.000
Airplane! (1980),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,5.000,0.000,4.000,0.000,0.000,3.500
Aladdin (1992),0.000,0.000,0.000,4.000,4.000,5.000,3.000,0.000,0.000,4.000,...,0.000,0.000,0.000,3.000,3.500,0.000,0.000,3.000,0.000,0.000
Alien (1979),4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,4.000,3.000,4.000,0.000,4.500
Aliens (1986),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,3.500,0.000,4.500,0.000,5.000


In [65]:
query_index = np.random.choice(movie_feature_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_feature_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors=5) # sebelumnya 10

57


In [71]:
movie_feature_df.index[57]

'Contact (1997)'

In [70]:
movie_feature_df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000
Addams Family Values (1993),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,4.000,0.000,4.000,0.000,0.000,0.000,2.500,0.000,0.000
Airplane! (1980),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,5.000,0.000,4.000,0.000,0.000,3.500
Aladdin (1992),0.000,0.000,0.000,4.000,4.000,5.000,3.000,0.000,0.000,4.000,...,0.000,0.000,0.000,3.000,3.500,0.000,0.000,3.000,0.000,0.000
Alien (1979),4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,4.000,3.000,4.000,0.000,4.500
Aliens (1986),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,3.500,0.000,4.500,0.000,5.000


In [67]:
# Pastikan variabel 'rating_popular_movie' sudah ada dan berisi data
movie_features_df = rating_popular_movie.pivot_table(
    index='title',
    columns='userId',
    values='rating'
).fillna(0) # Mengganti nilai NaN (kosong) dengan 0

# Tampilkan 5 baris pertama untuk memastikan hasilnya ada
print(movie_features_df.head())

userId                                  1     2     3     4     5     6    \
title                                                                       
2001: A Space Odyssey (1968)          0.000 0.000 0.000 0.000 0.000 0.000   
300 (2007)                            0.000 0.000 0.000 0.000 0.000 0.000   
Ace Ventura: Pet Detective (1994)     0.000 0.000 0.000 0.000 3.000 3.000   
Ace Ventura: When Nature Calls (1995) 0.000 0.000 0.000 0.000 0.000 2.000   
Addams Family Values (1993)           0.000 0.000 0.000 0.000 3.000 3.000   

userId                                  7     8     9     10   ...   601  \
title                                                          ...         
2001: A Space Odyssey (1968)          4.000 0.000 0.000 0.000  ... 0.000   
300 (2007)                            0.000 0.000 0.000 3.000  ... 0.000   
Ace Ventura: Pet Detective (1994)     0.000 0.000 0.000 0.000  ... 0.000   
Ace Ventura: When Nature Calls (1995) 0.000 0.000 0.000 0.000  ... 0.000   
Adda

In [68]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        # Mencetak judul film query (film yang dicari rekomendasinya)
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        # Mencetak film rekomendasi (dimulai dari i=1 karena i=0 adalah film itu sendiri)
        print('{0}: {1}, with distance of {2}'.format(
              i,
              movie_features_df.index[indices.flatten()[i]],
              distances.flatten()[i]))

Recommendations for Contact (1997):

1: Terminator, The (1984), with distance of 0.48511868715286255
2: Fifth Element, The (1997), with distance of 0.4977492690086365
3: Blade Runner (1982), with distance of 0.518856406211853
4: Star Wars: Episode I - The Phantom Menace (1999), with distance of 0.5189560651779175


In [72]:
movie_features_df.index[50:70]

Index(['Chicken Run (2000)', 'City of God (Cidade de Deus) (2002)',
       'Clear and Present Danger (1994)', 'Clerks (1994)',
       'Cliffhanger (1993)', 'Clockwork Orange, A (1971)', 'Clueless (1995)',
       'Contact (1997)', 'Crimson Tide (1995)',
       'Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)',
       'Dances with Wolves (1990)', 'Dark Knight Rises, The (2012)',
       'Dark Knight, The (2008)', 'Dave (1993)', 'Dead Poets Society (1989)',
       'Demolition Man (1993)', 'Departed, The (2006)', 'Die Hard (1988)',
       'Die Hard: With a Vengeance (1995)', 'Dogma (1999)'],
      dtype='object', name='title')

In [73]:
movie_features_df.index[57]

'Contact (1997)'